<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/03_embeddings_sherlock_holmes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Embeddings de Palabras usando Sherlock Holmes


# 1) Carga de Datos

Carga un texto desde Project Gutenberg y elimina el header/footer:

In [1]:
import urllib.request
from pathlib import Path

def load_gutenberg_text(url):
    print("Cargando texto desde:", url, "..")
    response = urllib.request.urlopen(url)
    raw = response.read().decode('utf-8')

    # Encontrar el inicio y fin del contenido real (eliminar header/footer de Gutenberg)
    start = raw.find("*** START OF THE PROJECT GUTENBERG")
    start = raw.find("\n", start) + 1
    end = raw.find("*** END OF THE PROJECT GUTENBERG")

    return raw[start:end]

# URLs de los libros en Project Gutenberg
urls = [
    "https://www.gutenberg.org/files/1661/1661-0.txt",    # The Adventures of Sherlock Holmes
    "https://www.gutenberg.org/files/108/108-0.txt",      # The Return of Sherlock Holmes
    "https://www.gutenberg.org/files/2097/2097-0.txt",    # The Hound of the Baskervilles
    "https://www.gutenberg.org/files/244/244-0.txt"       # A Study in Scarlet
]

# Carga y guarda el texto
text = " "
for url in urls:
  book_text = load_gutenberg_text(url)
  text += book_text


print("Texto cargado:", len(text), "caracteres")

Cargando texto desde: https://www.gutenberg.org/files/1661/1661-0.txt ..
Cargando texto desde: https://www.gutenberg.org/files/108/108-0.txt ..
Cargando texto desde: https://www.gutenberg.org/files/2097/2097-0.txt ..
Cargando texto desde: https://www.gutenberg.org/files/244/244-0.txt ..
Texto cargado: 1742150 caracteres


# 2) Preprocesamiento

Tokeniza y limpia el texto:
* Convierte a minúsculas
* Elimina tokens que no son palabras

In [2]:
import spacy
from spacy.lang.en import English

# Inicializar spaCy (solo tokenizador para velocidad)
nlp = English(disable=['tagger','parser','ner'])

def tokenize(text):
    doc = nlp(text)
    return [token.text.lower() for token in doc
            if token.text.strip() and not token.is_punct]

# Dividir en oraciones y tokenizar
corpus_sentences = []
for line in text.split('\n'):
    if line.strip():  # ignorar líneas vacías
        tokens = tokenize(line)
        if tokens:  # ignorar líneas sin tokens válidos
            corpus_sentences.append(tokens)

print("Total de oraciones procesadas:", len(corpus_sentences))

Total de oraciones procesadas: 27683


# 3) Entrenamiento de Modelos

In [3]:
from gensim.models import Word2Vec, FastText

Entrenar Word2Vec

In [4]:
w2v_model = Word2Vec(sentences=corpus_sentences,
                    vector_size=300,      # Aumentar dimensionalidad (300)
                    window=8,             # Ventana más amplia para capturar más contexto (8)
                    min_count=2,          # Filtrar palabras poco frecuentes (5)
                    workers=4,
                    sg=1,                 # Usar Skip-gram
                    epochs= 30,          # Más épocas de entrenamiento
                    negative= 15,        # Negative sampling
                    alpha= 0.025,        # Learning rate inicial
                    min_alpha= 0.0001    # Learning rate final
                  )

Entrenar FastText

In [5]:
ft_model = FastText(sentences=corpus_sentences,
                   vector_size=300,    # Aumentar dimensionalidad (300)
                   window=8,           # Ventana más amplia para capturar más contexto (8)
                   min_count=2,        # Mantener min_count bajo para capturar más variantes
                   workers=4,
                   sg=1,               # Skip-gram para mejor calidad
                   min_n=2,            # Tamaño mínimo de n-gramas
                   max_n=6,            # Tamaño máximo de n-gramas (aumentado para capturar más patrones)
                   epochs=30,          # Más épocas de entrenamiento
                   word_ngrams=1,      # Habilitar n-gramas de palabras
                   negative=15,        # Más muestras negativas
                   alpha=0.025,        # Learning rate inicial
                   min_alpha=0.0001    # Learning rate final
              )

Guardar modelos

In [6]:
w2v_model.save("sherlock_w2v.model")
ft_model.save("sherlock_ft.model")

# 4) Análisis de Similitudes

Cargar vectores

In [7]:
w2v_vectors = w2v_model.wv
ft_vectors = ft_model.wv

print("\nEstadísticas de los modelos:")
print("Dimensión de los vectores:", w2v_vectors.vector_size)
print("Número de palabras (Word2Vec):", len(w2v_vectors.index_to_key))
print("Número de palabras (FastText):", len(ft_vectors.index_to_key))


Estadísticas de los modelos:
Dimensión de los vectores: 300
Número de palabras (Word2Vec): 8416
Número de palabras (FastText): 8416


In [8]:
print("\nPalabras más similares a 'holmes' (Word2Vec):")
print(w2v_vectors.most_similar('holmes'))


Palabras más similares a 'holmes' (Word2Vec):
[('sherlock', 0.5627038478851318), ('demurely', 0.4705038368701935), ('gleefully', 0.4312404692173004), ('involuntarily', 0.4271424114704132), ('cheerily', 0.4240522086620331), ('sardonically', 0.4198615252971649), ('misjudged', 0.41461366415023804), ('compel', 0.41146203875541687), ('aback', 0.4071274995803833), ('triumphantly', 0.4049912095069885)]


In [9]:
print("\nPalabras más similares a 'crime' (Word2Vec):")
print(w2v_vectors.most_similar('crime'))



Palabras más similares a 'crime' (Word2Vec):
[('committed', 0.5095756649971008), ('literature', 0.47296348214149475), ('featureless', 0.4724273979663849), ('deliberate', 0.45530077815055847), ('records', 0.45302456617355347), ('logic', 0.4432651400566101), ('detect', 0.43559083342552185), ('talent', 0.4308238923549652), ('perpetrator', 0.4281263053417206), ('sots', 0.4210221469402313)]


In [15]:
print("\nSimilitud entre 'crime' y 'art':")
print("Word2Vec:", w2v_vectors.similarity('crime', 'art'))
print("FastText:", ft_vectors.similarity('crime', 'art'))


Similitud entre 'crime' y 'art':
Word2Vec: 0.22276348
FastText: 0.073123716


# 5) Experimentos con palabras fuera de vocabulario

In [11]:
print("\nPrueba con palabra fuera de vocabulario:")
try:
    print("Word2Vec - Similares a 'investigador':")
    print(w2v_vectors.most_similar('investigador'))
except KeyError:
    print("Word2Vec no puede manejar palabras fuera de vocabulario")

print("\nFastText - Similares a 'investigador':")
print(ft_vectors.most_similar('investigador'))  # FastText puede generar vectores para palabras nuevas



Prueba con palabra fuera de vocabulario:
Word2Vec - Similares a 'investigador':
Word2Vec no puede manejar palabras fuera de vocabulario

FastText - Similares a 'investigador':
[('investigate', 0.9296735525131226), ('investigated', 0.915936291217804), ('investigating', 0.9112592935562134), ('investigations', 0.8825503587722778), ('investigation', 0.8762537240982056), ('invest', 0.8713880777359009), ('investments', 0.7584698796272278), ('testimonial', 0.5811780691146851), ('domestic', 0.5748422145843506), ('obligations', 0.5539227724075317)]


# 6) Analogías

In [17]:
print("\nAnalogías (Word2Vec):")
result = w2v_vectors.most_similar(positive=['holmes', 'police'],
                                negative=['evidence'])
print("holmes:police como crime:?")
print(result)


Analogías (Word2Vec):
holmes:police como crime:?
[('sherlock', 0.3552018105983734), ('wink', 0.337390661239624), ('jove', 0.3190644085407257), ('duncan', 0.309001088142395), ('soda', 0.30734172463417053), ('triumph', 0.30677905678749084), ('apologize', 0.3051346242427826), ('grinning', 0.3048025369644165), ('curtly', 0.3041435480117798), ('basket', 0.2993147075176239)]
